In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import optuna
from tqdm import tqdm
import seaborn as sns;

In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Concatenate, Reshape, GlobalMaxPooling2D, GlobalMaxPooling3D, GlobalAveragePooling3D, Conv2D, Conv1D, Add
import random
import sklearn
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from keras import optimizers
from sklearn.utils import class_weight
from tensorflow.keras.optimizers import SGD, Adam
import cv2
import albumentations as A
from keras.models import Sequential, Model
from keras.models import model_from_json
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
from keras.applications.densenet import DenseNet169, DenseNet121, DenseNet201
from keras.applications.efficientnet import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from keras.applications.xception import Xception

2024-05-18 13:47:04.558969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 13:47:05.214317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/anaconda3/envs/matheus_levy_tensorflow/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:
2024-05-18 13:47:05.214388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object fil

In [3]:
def resize(width, height, img):
  altura, largura, _ = img.shape

  imagem_redimensionada = img
  if (largura > width):
    nova_largura = width
    altura_original, largura_original = img.shape[:2]
    nova_altura = int((nova_largura / largura_original) * altura_original)
    imagem_redimensionada = cv2.resize(img, (nova_largura, nova_altura))
    altura, largura, _ = imagem_redimensionada.shape
 

  if (altura > height):
    nova_altura = height
    altura_original, largura_original = imagem_redimensionada.shape[:2]
    nova_largura = int((nova_altura / altura_original) * largura_original)
    imagem_redimensionada = cv2.resize(imagem_redimensionada, (nova_largura, nova_altura))


  delta_largura = width - imagem_redimensionada.shape[1]
  delta_altura = height - imagem_redimensionada.shape[0]

  # Calcular o preenchimento necessário em cada lado da imagem
  top = delta_altura // 2
  bottom = delta_altura - top
  left = delta_largura // 2
  right = delta_largura - left

  # Adicionar preenchimento à imagem
  img_com_preenchimento = cv2.copyMakeBorder(imagem_redimensionada, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
  return img_com_preenchimento

def get_json_from_img(filename, path):
  filename =  os.path.splitext(filename)[0]
  filename += '.json'
  with open(os.path.join(path, 'labels', filename), 'r') as openfile:
    json_object = json.load(openfile)
  return json_object

def read_dataset(path):
  x, y = [], []
  for file in tqdm(os.listdir(os.path.join(path, 'image'))):
    img = cv2.imread(os.path.join(path, 'image', file))[...,::-1]
    img = resize(128, 128, img)
    ann = get_json_from_img(file, path)
    label = ann['category_id']
    # if label != -1:
    x.append(img)
    y.append(label)
  return np.array(x), np.array(y)


In [4]:
x_train, y_train = read_dataset('/home/matheus_levy/workspace/dataset/Crop_dataset/train')
x_val, y_val = read_dataset('/home/matheus_levy/workspace/dataset/Crop_dataset/val')
x_test, y_test = read_dataset('/home/matheus_levy/workspace/dataset/Crop_dataset/test')

100%|██████████| 2228/2228 [00:06<00:00, 327.62it/s]


In [5]:
x_train = np.load('/home/matheus_levy/workspace/crops_npy/x_train.npy')
y_train = np.load('/home/matheus_levy/workspace/crops_npy/y_train.npy')

x_val = np.load('/home/matheus_levy/workspace/crops_npy/x_val.npy')
y_val = np.load('/home/matheus_levy/workspace/crops_npy/y_val.npy')

x_test = np.load('/home/matheus_levy/workspace/crops_npy/x_test.npy')
y_test = np.load('/home/matheus_levy/workspace/crops_npy/y_test.npy')

In [7]:
print(np.unique(y_train))
print(np.unique(y_val))
print(np.unique(y_test))

[ 1  2  3  4  5  6  7  8  9 10 11]
[ 1  2  3  4  5  6  7  8  9 10 11]
[ 1  2  3  4  5  6  7  8  9 10 11]


In [8]:
y_train-=1
y_val-=1
y_test-=1

In [9]:
print(np.unique(y_train))
print(np.unique(y_val))
print(np.unique(y_test))

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


In [4]:
model = tf.keras.models.load_model('/home/matheus_levy/workspace/classifiers/densenet201.keras')

2024-05-18 13:47:20.280029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-18 13:47:20.312589: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-18 13:47:20.313053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-18 13:47:20.313933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [6]:
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap="viridis")
    plt.show()
    print(sklearn.metrics.classification_report(true, preds, digits=5))

In [8]:
def get_ann_from_image_id(id, coco_preds):
  preds = [pred for pred in coco_preds if pred['image_id'] == id]
  return preds

def match_ann_bbox(bbox, anns_rpn):
  for ann_rpn in anns_rpn:
    if ann_rpn['bbox'] == bbox:
      return ann_rpn

# Model prediction
# Get the crop (image) and get the label (json)
# The prediction label always has category_id == 0 which indicates an object there
# Resize the crop (image) and model prediction
# Go to the RPN result json (result json)
# Get all predictions from the image_id of the current label
# Check which one has the same bbox
# Change the category_id of this annotation to the predicted category_id
crop_path =  '/home/matheus_levy/workspace/dataset/Crops/Cascade_R_CNN/val'
path_detection_1_stage = '/home/matheus_levy/workspace/RPN_YOLO_Center_Retina/Cascade_R_CNN/predicts_bbox_cascade_swin_val.json.bbox.json'
y_pred = [] 
y_true = []
final_pred = []
for filename in tqdm(os.listdir(os.path.join(crop_path, 'images'))):
  img = cv2.imread(os.path.join(crop_path, 'images', filename))[...,::-1]
  img_crop= resize(128, 128, img)
  ann_crop = get_json_from_img(filename, crop_path)
  bbox_crop = ann_crop['bbox']
  pred_class = model(np.expand_dims(img_crop/255.0, axis=0))
  pred_class = np.argmax(pred_class)
  y_pred.append(pred_class)
  y_true.append(ann_crop['category_id'])
  with open(path_detection_1_stage, 'r') as openfile:
    coco_pred_rpn = json.load(openfile)
  image_id_crop = ann_crop['image_id']
  predicts_rpn_anns_for_image_id = get_ann_from_image_id(image_id_crop, coco_pred_rpn)
  ann_match = match_ann_bbox(bbox_crop, predicts_rpn_anns_for_image_id)
  ann_match['category_id'] = int(pred_class) + 1
  final_pred.append(ann_match)

with open("/home/matheus_levy/workspace/test.json", "w") as outfile:
  json.dump(final_pred, outfile, indent=2)

 21%|██        | 554/2625 [02:08<07:47,  4.43it/s]

In [ ]:
draw_confusion_matrix(y_true,y_pred)

In [33]:
with open(path_detection_1_stage, 'r') as openfile:
    coco_pred_rpn = json.load(openfile)
print(len(coco_pred_rpn))

2260


In [34]:
len(final_pred)

2260

In [61]:
crop_path = '/home/matheus_levy/workspace/dataset/CenterNet_Crop'
x_test, y_test = read_dataset(crop_path)

100%|██████████| 2225/2225 [00:07<00:00, 314.82it/s]


In [62]:
y_test.shape

(2225,)

In [63]:
len(y_pred)

2225

In [25]:
# 1. Pegar predições
# 2. Limite de Score (>=0.5)
# 3. Verificar IoU com o GT
# 4. Pegar a classe com o GT
# 5. Crop da bbox com +10% de expansão
# 6. Salvar o crop como imagem e a classe no txt
import json
import cv2
import os 
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from tqdm import tqdm

# 'train' mode to generate crops to train a classifier
# 'pred' mode to generate crops to be classificate by a classifier (do not generate the groundtruth label)
mode= 'pred'

def get_prediction_from_coco_json(json_path, score_thr=0.0):
    with open(json_path, 'r') as openfile:
        coco_json_result = json.load(openfile)
    if mode == 'train':
        return coco_json_result['annotations']
    else:
        return [pred for pred in coco_json_result if pred['score'] > score_thr]

def get_coco_json(gt_path=r"/home/matheus_levy/workspace/dataset/mdetection_dataset/test/1_class_annotation.coco.json"):
    with open(gt_path, 'r') as openfile:
        coco_json_gt = json.load(openfile)
    return coco_json_gt

def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1[0], box1[1], box1[2], box1[3]
    x2, y2, w2, h2 = box2[0], box2[1], box2[2], box2[3]

    x_inter = max(x1, x2)
    y_inter = max(y1, y2)
    w_inter = max(0, min(x1 + w1, x2 + w2) - x_inter)
    h_inter = max(0, min(y1 + h1, y2 + h2) - y_inter)

    area_inter = w_inter * h_inter
    area_union = w1 * h1 + w2 * h2 - area_inter

    iou = area_inter / area_union
    return iou

def get_ann_from_image_id(id, coco_preds, coco_gts):
    preds = [pred for pred in coco_preds if pred['image_id'] == id]
    if mode == 'pred':
        gts = [gt for gt in coco_gts if gt['image_id'] == id]
    else:
        gts = None
    return preds, gts

def match_preds_with_gt(preds, gts, iou_thr=0.5):
    for pred in preds:
        pred_bbox = pred['bbox']
        best_iou = 0
        best_ann = None
        for gt in gts:
            gt_bbox = gt['bbox']
            iou = calculate_iou(pred_bbox, gt_bbox)
            if iou > best_iou:
                best_iou = iou
                best_ann = gt
        if best_ann is not None:
            pred['category_id'] = best_ann['category_id']
        else:
            pred['category_id'] = -1
    return preds

def qtd_imagens_in_results(coco_preds):
    image_ids = set()
    for prediction in coco_preds:
        image_id = prediction['image_id']
        image_ids.add(image_id)
    return len(image_ids)

def qtd_imagens_in_gt(coco_gt):
    imagens = coco_gt['images']
    return len(imagens)


def image_id_to_name(id, coco_gt):
    imagens = coco_gt['images']
    for imagem in imagens:
        if imagem['id'] == id:
            return imagem['file_name']

def imagem_by_id(id, coco_gt, path_to_image):
    file_name = image_id_to_name(id, coco_gt)
    img = cv2.imread(os.path.join(path_to_image, file_name))
    return img

def crop_imagem(imagem, bbox, aumento=0.3):
    x,y, largura, altura = bbox[0], bbox[1], bbox[2], bbox[3]
    aumento_largura = largura * aumento
    aumento_altura = altura * aumento
    x -= aumento_largura / 2  # Subtrai metade do aumento da largura do ponto x
    y -= aumento_altura / 2   # Subtrai metade do aumento da altura do ponto y
    if (x<0):
        x=0
    if (y<0):
        y=0
    largura += aumento_largura  # Aumenta a largura
    altura += aumento_altura    # Aumenta a altura
    x,y, largura, altura = int(x), int(y), int(largura), int(altura)
    return imagem[y:y+altura, x:x+largura]
    
json_path = r'/home/matheus_levy/workspace/RPN_YOLO_Center_Retina/Cascade_R_CNN/predicts_bbox_cascade_swin.json.bbox.json'
save_crop_image_path = r"/home/matheus_levy/workspace/dataset/Crops/Cascade_R_CNN/test/images"
save_crop_label_path = r"/home/matheus_levy/workspace/dataset/Crops/Cascade_R_CNN/test/labels"

preds = get_prediction_from_coco_json(json_path) # Durante o treino é o proprio gt
gts = get_coco_json()
if mode == 'pred':
    qtd_imagens = qtd_imagens_in_gt(gts)
if mode == 'train':
    qtd_imagens = len(preds)
preds_classes = []

for i in range(qtd_imagens):
    p, g = get_ann_from_image_id(i, preds, gts['annotations'])
    if mode == 'pred':
        ps = match_preds_with_gt(p, g)
        preds_classes.extend(ps)
    else:
        preds_classes.extend(p)

with open("retina_without_thr.json", "w") as outfile:
    json.dump(preds_classes, outfile, indent=2)

i=0
for pred in tqdm(preds_classes):
    imagem = imagem_by_id(pred['image_id'],
    gts, path_to_image=r"/home/matheus_levy/workspace/dataset/mdetection_dataset/test")
    crop = crop_imagem(imagem, pred['bbox'])
    if crop.size == 0:
        continue
    coco_bbox = pred["bbox"]
    x, y, largura, altura = coco_bbox

    cv2.imwrite(os.path.join(save_crop_image_path, f"{i}.png"), crop)
    if mode == 'train':
        json_label = {"image_id": pred['image_id'], 
                  "category_id": pred["category_id"],
                  "bbox": pred["bbox"], 
                  } 
    else:
        json_label = {"image_id": pred['image_id'], 
                    "category_id": pred["category_id"],
                    "bbox": pred["bbox"],
                    "score": pred["score"]
                    } 
    with open(f"{save_crop_label_path}/{i}.json", "w") as outfile:
        json.dump(json_label, outfile, indent=2)
    i+=1

100%|██████████| 2686/2686 [03:11<00:00, 14.03it/s]
